In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
data = pd.read_csv("model_df.csv")
data.head(5)

,paragraph,participants,file_name,date,company_paticipants_yes,other_paticipants_yes,company_name,ticker,Close,D-1,...,topic_74,topic_75,topic_76,topic_77,topic_78,topic_79,topic_80,topic_81,topic_82,topic_83
0,I am Hara from NKSJ Holdings. Thank you very m...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.0,0.038436,...,0.002155,0.003115,0.002470,0.002397,0.001761,0.004951,0.002397,0.002489,0.003408,0.003206
1,sure you're tired by now at this late hour but...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.0,0.038436,...,0.002155,0.003115,0.002470,0.002397,0.001761,0.004951,0.002397,0.002489,0.003408,0.003206
2,What I am going to explain today is only the f...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.0,0.038436,...,0.002459,0.003555,0.002819,0.002736,0.002010,0.005650,0.002736,0.002841,0.003889,0.003660
3,full fiscal year of 2012 and shareholder retur...,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.0,0.038436,...,0.002155,0.003115,0.002470,0.002397,0.001761,0.004951,0.002397,0.002489,0.003408,0.003206
4,"NKSJ Group management plan announced today, Pr...",Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,0,Sompo Holdings,8630.T,1594.0,0.038436,...,0.001571,0.002271,0.001801,0.001748,0.001284,0.003610,0.001748,0.001815,0.002485,0.002338


In [ ]:
df = data.copy()

#create list of columns name
col_topic = []
for i in range(1,84):
  name ='topic_'+str(i)
  col_topic.append(name) 

columns = col_topic.copy()
columns.append('paragraph')
columns.append('file_name')
#columns.append('ticker')

# take only necessary columns
df_prob_topic = df[columns].copy()
# calculate len of each para
df_prob_topic['len_para'] = df_prob_topic['paragraph'].copy().apply(lambda r: len(r))
# calculate len of each file
df_len_file = df_prob_topic[['len_para','file_name']].copy()
df_len_file = df_len_file.groupby('file_name').sum().reset_index()
df_len_file = df_len_file.rename(columns = {'len_para':'len_file'})
# join back to prob df
df_join = df_prob_topic.join(df_len_file.set_index('file_name'), on='file_name')
# calculate weight
df_join['weight']=df_join['len_para']/df_join['len_file']
# calculate weighted of each topic
df_weight = pd.DataFrame()
for col in col_topic:
  df_weight[col] = df_join[col]*df_join['weight']
df_weight['file_name'] = df_join['file_name']
# calculate weighted average
df_wa = df_weight.groupby('file_name').sum().reset_index()
df_wa.head(2)
#df_wa.to_csv('WA_per_file.csv')

,file_name,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_74,topic_75,topic_76,topic_77,topic_78,topic_79,topic_80,topic_81,topic_82,topic_83
0,20110227_QBE_INSURANCE-_Earnings_Call_2011-2-2...,0.019923,0.006651,0.008153,0.011782,0.024530,0.010107,0.012335,0.008410,0.007138,...,0.008433,0.011987,0.006948,0.004342,0.004134,0.034347,0.007320,0.008235,0.007772,0.007837
1,20110404_QBE_INSURANCE-_Shareholder_Mtg_Call_2...,0.015977,0.012147,0.010328,0.005594,0.028684,0.006463,0.007683,0.007308,0.028016,...,0.016717,0.013265,0.004850,0.012357,0.006479,0.031200,0.009174,0.009899,0.009509,0.009612


In [ ]:
#create a copy
df = data.copy()

#define category
cat_1 = ['8630.T', '8725.T', '8766.T'] 
cat_2 = ['QBE.AX','SREN.SW', 'SCR.PA', 'HNR1.DE', 'MUV2.DE']
def find_category(ticker):
  if ticker in cat_1:
    category = 'Japanese & Pacific'
  elif ticker in cat_2:
    category = 'European (Re)Insurers'
  return category
df['category']=df['ticker'].apply(lambda r: find_category(r))

# define year
def find_year(date):
  year = date[0:4]
  return year
df['year']=df['date'].apply(lambda r: find_year(r))

#create list of columns name
col_topic = []
for i in range(1,84):
  name ='topic_'+str(i)
  col_topic.append(name) 

columns = col_topic.copy()
columns.append('paragraph')
columns.append('category')
columns.append('year')


# take only necessary columns
df_prob_topic = df[columns].copy()
# calculate len of each para
df_prob_topic['len_para'] = df_prob_topic['paragraph'].copy().apply(lambda r: len(r))
# calculate len of each category
df_len_ctg = df_prob_topic[['len_para','category']].copy()
df_len_ctg = df_len_ctg.groupby('category').sum().reset_index()
df_len_ctg = df_len_ctg.rename(columns = {'len_para':'len_category'})
# join back to prob df
df_join = df_prob_topic.join(df_len_ctg.set_index('category'), on='category')
# calculate weight
df_join['weight']=df_join['len_para']/df_join['len_category']
# calculate weighted of each topic
df_weight = pd.DataFrame()
for col in col_topic:
  df_weight[col] = df_join[col]*df_join['weight']
df_weight['category'] = df_join['category']
df_weight['year'] = df_join['year']
# calculate weighted average
df_wa = df_weight.groupby(['category','year']).sum().reset_index()
df_wa.head(2)

,category,year,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_74,topic_75,topic_76,topic_77,topic_78,topic_79,topic_80,topic_81,topic_82,topic_83
0,European (Re)Insurers,2011,0.000204,0.000107,0.000105,0.000098,0.000302,0.000094,0.000113,0.000089,...,0.000143,0.000143,0.000067,0.000095,0.000060,0.000372,0.000094,0.000103,0.000098,0.000099
1,European (Re)Insurers,2012,0.000311,0.000123,0.000321,0.000141,0.000233,0.000203,0.000243,0.000129,...,0.000183,0.000190,0.000119,0.000078,0.000076,0.000714,0.000180,0.000122,0.000141,0.000176


In [ ]:
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
df_eu = df_wa.loc[df_wa['category']=='European (Re)Insurers']
df_eu_plot = df_eu.drop('category', axis = 1)
df_eu_melt = df_eu_plot.melt(id_vars=['year'], 
                            var_name="topic", 
                            value_name="value")
print(df_eu_melt.head(5))

fig_1 = px.line(df_eu_melt, x="year", y="value", color='topic',title = 'European (Re)Insurers')
fig_1.show()

   year    topic     value
0  2011  topic_1  0.000204
1  2012  topic_1  0.000311
2  2013  topic_1  0.000204
3  2014  topic_1  0.000745
4  2015  topic_1  0.001515


For European (Re)Insurers
1.   in 2012: topic 79
2.   in 2015: topic 46,5, 37
3.   in 2016: topic 46,54,19
4.   in 2017: topic 46
5.   mid 2017: 46,79
6.   2018: 79,36,46
7.   2019: 79, 46
8.   2020, 2021 : 79, 46,54



In [ ]:
df_jp = df_wa.loc[df_wa['category']=='Japanese & Pacific']
df_jp_plot = df_jp.drop('category', axis = 1)
df_jp_melt = df_jp_plot.melt(id_vars=['year'], 
                            var_name="topic", 
                            value_name="value")
print(df_jp_melt.head(5))

fig_2 = px.line(df_jp_melt, x="year", y="value", color='topic', title = 'Japanese & Pacific')
fig_2.show()

   year    topic     value
0  2011  topic_1  0.001452
1  2012  topic_1  0.003603
2  2013  topic_1  0.004785
3  2014  topic_1  0.005438
4  2015  topic_1  0.003953


For Japanese & Pacific
0.   in 2011: topic 1, 17, 39
1.   in 2012: topic 1,39
2.   in 2013: topic 1,39,40
3.   in 2014: topic 1,39,40, 56
4.   in 2015, 2016: topic 39, 1; in 2015, topic 33 have higher prob compare to 2016; Other are the same in trend
5.   in 2017: 1,39,26,40
6.   in 2018, 2019: 1,39,29
7.   in 2020: topic 1
8.   in 2021: topic 2


In [ ]:
fig_1.write_html("European.html")
fig_2.write_html("Japan.html")